## Load NAICS/ISIC/SIC code concordance table data

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

### We have local copies rooted in the S3_BUCKET : https://redhat-osc-physical-landing-647521352890.s3.amazonaws.com/USSIC-ISIC-NAICS/

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

## Environment variables and dot-env

The following cell looks for a "dot-env" file in some standard locations,
and loads its contents into `os.environ`.

In [1]:
import os
import pathlib
from dotenv import load_dotenv

# Load some standard environment variables from a dot-env file, if it exists.
# If no such file can be found, does not fail, and so allows these environment vars to
# be populated in some other way
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

## S3 and boto3

In [2]:
import boto3
s3_src = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
src_bucket = s3_src.Bucket(os.environ['S3_LANDING_BUCKET'])

## Connecting to Trino with sqlalchemy

In [3]:
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ['TRINO_USER'],
    host = os.environ['TRINO_HOST'],
    port = os.environ['TRINO_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ['TRINO_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev',
}

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'

engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

## Load Data Dictionary for US Census 2017 All-sector report

In [4]:
import pandas as pd

isic3_sic = 'ISIC-USSIC.csv'
bObj = src_bucket.Object(f'USSIC-ISIC-NAICS/{isic3_sic}')
bObj.download_file(f'/tmp/ingest-{isic3_sic}')

# We drop the fact of partial concordance and the descriptions of what's inside/outside the overlap.
# For our purposes, if there's any overlap, we take it.
isic3_sic_df = pd.read_csv(f'/tmp/ingest-{isic3_sic}', sep=',', header=0,
                           usecols=['ISIC3', 'US-SIC'],
                           engine='c')
# dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{isic3_sic}')
isic3_sic_df

,ISIC3,US-SIC
0,111,111
1,111,112
2,111,115
3,111,116
4,111,119
...,...,...
1705,8520,751
1706,2720,3492
1707,3311,5661
1708,121,219


In [5]:
isic4_isic31 = 'ISIC4_ISIC31.txt'
bObj = src_bucket.Object(f'USSIC-ISIC-NAICS/{isic4_isic31}')
bObj.download_file(f'/tmp/ingest-{isic4_isic31}')

# We drop the fact of partial concordance and the descriptions of what's inside/outside the overlap.
# For our purposes, if there's any overlap, we take it.
isic4_isic31_df = pd.read_csv(f'/tmp/ingest-{isic4_isic31}', sep=',', header=0,
                              usecols=['ISIC4code', 'ISIC31code'],
                              engine='c')
# dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{isic4_isic31}')
isic4_isic31_df

,ISIC4code,ISIC31code
0,111,111
1,112,111
2,113,111
3,113,112
4,114,111
...,...,...
732,9609,9309
733,9700,9500
734,9810,9600
735,9820,9700


In [6]:
naics2012_isic4 = 'NAICS2012US-ISIC4.txt'
bObj = src_bucket.Object(f'USSIC-ISIC-NAICS/{naics2012_isic4}')
bObj.download_file(f'/tmp/ingest-{naics2012_isic4}')

# We drop the fact of partial concordance and the descriptions of what's inside/outside the overlap.
# For our purposes, if there's any overlap, we take it.
naics2012_isic4_df = pd.read_csv(f'/tmp/ingest-{naics2012_isic4}', sep=',', header=0,
                                 usecols=['NAICS2012Code', 'ISIC4Code'],
                                 engine='c')
naics2012_isic4_df['NAICS2012Code'] = naics2012_isic4_df['NAICS2012Code'].astype('Int64')
# dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{naics2012_isic4}')
naics2012_isic4_df

,NAICS2012Code,ISIC4Code
0,111110,111
1,111120,111
2,111130,111
3,111140,111
4,111150,111
...,...,...
1660,928120,8421
1661,928120,8423
1662,928120,9900
1663,<NA>,9810


In [7]:
concordance_2012_df = (naics2012_isic4_df
 # .drop(columns=['NAICS2012Part', 'ISIC4Part'])
 .merge(isic4_isic31_df, left_on='ISIC4Code', right_on='ISIC4code')
 # .drop(columns=['partialISIC4','partialISIC31'])
 .merge(isic3_sic_df, left_on='ISIC31code', right_on='ISIC3')
 # .drop(columns=['ISIC-partial','USSIC_partial'])
)
concordance_2012_df

,NAICS2012Code,ISIC4Code,ISIC4code,ISIC31code,ISIC3,US-SIC
0,111110,111,111,111,111,111
1,111110,111,111,111,111,112
2,111110,111,111,111,111,115
3,111110,111,111,111,111,116
4,111110,111,111,111,111,119
...,...,...,...,...,...,...
34114,926110,8412,8412,7512,7512,9531
34115,923130,8430,8430,7530,7530,6321
34116,923130,8430,8430,7530,7530,6324
34117,923130,8430,8430,7530,7530,9441


In [8]:
from zipfile import ZipFile

naics2017_isic4 = 'NAICS2017US-ISIC4.zip'
bObj = src_bucket.Object(f'USSIC-ISIC-NAICS/{naics2017_isic4}')
bObj.download_file(f'/tmp/ingest-{naics2017_isic4}')
files = ZipFile(f'/tmp/ingest-{naics2017_isic4}')

# We drop the fact of partial concordance and the descriptions of what's inside/outside the overlap.
# For our purposes, if there's any overlap, we take it.
naics2017_isic4_df = pd.read_excel(files.open("2017_NAICS_to_ISIC_4.xlsx"), header=0)
naics2017_isic4_df.columns = naics2017_isic4_df.columns.map(lambda x: x.replace('\n', ' ').rstrip())
naics2017_isic4_df = naics2017_isic4_df.drop(columns=['Part of NAICS US', 'Part of ISIC'])
# dd_df.columns = ['name', 'desc'] 

os.unlink(f'/tmp/ingest-{naics2017_isic4}')
naics2017_isic4_df

/Users/michael/miniconda3/envs/timestamp/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,2017 NAICS US,2017 NAICS US TITLE,ISIC 4.0,ISIC Revision 4.0 Title,"Notes: link content based on NAICS definition, entire NAICS industry if blank"
0,0,Multiple NAICS industries,9810,Undifferentiated goods-producing activities of...,one to many links for ISIC 9810 to all of NAIC...
1,0,Multiple NAICS industries,9820,Undifferentiated service-producing activities ...,one to many links for ISIC 9820 to all of NAIC...
2,111110,Soybean Farming,111,"Growing of cereals (except rice), leguminous c...",
3,111120,Oilseed (except Soybean) Farming,111,"Growing of cereals (except rice), leguminous c...",
4,111130,Dry Pea and Bean Farming,111,"Growing of cereals (except rice), leguminous c...",
...,...,...,...,...,...
1650,928110,National Security,8423,Public order and safety activities,"military police, coast guard, and similar"
1651,928110,National Security,8423,Public order and safety activities,military courts
1652,928120,International Affairs,8421,Foreign affairs,except immigration and border police and extra...
1653,928120,International Affairs,8423,Public order and safety activities,immigration and border police


In [9]:
concordance_2017_df = (naics2017_isic4_df
 # .drop(columns=['Part of NAICS US', 'Part of ISIC'])
 .merge(isic4_isic31_df, left_on='ISIC 4.0', right_on='ISIC4code')
 # .drop(columns=['partialISIC4','partialISIC31'])
 .merge(isic3_sic_df, left_on='ISIC31code', right_on='ISIC3')
 # .drop(columns=['ISIC-partial','USSIC_partial'])
)
concordance_2017_df

,2017 NAICS US,2017 NAICS US TITLE,ISIC 4.0,ISIC Revision 4.0 Title,"Notes: link content based on NAICS definition, entire NAICS industry if blank",ISIC4code,ISIC31code,ISIC3,US-SIC
0,111110,Soybean Farming,111,"Growing of cereals (except rice), leguminous c...",,111,111,111,111
1,111110,Soybean Farming,111,"Growing of cereals (except rice), leguminous c...",,111,111,111,112
2,111110,Soybean Farming,111,"Growing of cereals (except rice), leguminous c...",,111,111,111,115
3,111110,Soybean Farming,111,"Growing of cereals (except rice), leguminous c...",,111,111,111,116
4,111110,Soybean Farming,111,"Growing of cereals (except rice), leguminous c...",,111,111,111,119
...,...,...,...,...,...,...,...,...,...
34039,926110,Administration of General Economic Programs,8412,Regulation of the activities of providing heal...,administration of art and cultural programs,8412,7512,7512,9531
34040,923130,Administration of Human Resource Programs (exc...,8430,Compulsory social security activities,"social security, workers compensation, unemplo...",8430,7530,7530,6321
34041,923130,Administration of Human Resource Programs (exc...,8430,Compulsory social security activities,"social security, workers compensation, unemplo...",8430,7530,7530,6324
34042,923130,Administration of Human Resource Programs (exc...,8430,Compulsory social security activities,"social security, workers compensation, unemplo...",8430,7530,7530,9441


In [10]:
engine.execute(f"drop table if exists {ingest_schema}.naics_isic_sic").fetchall()

[(True,)]

In [11]:
concordance_2012_df['naics_version'] = 2012
concordance_2012_df = (concordance_2012_df[['NAICS2012Code', 'naics_version', 'ISIC4code', 'US-SIC']]
                       .rename(columns={'NAICS2012Code':'naics_code', 'ISIC4code':'isic4_code', 'US-SIC':'sic'}))

In [12]:
concordance_2017_df['naics_version'] = 2017
concordance_2017_df = (concordance_2017_df[['2017 NAICS US', 'naics_version', 'ISIC 4.0', 'US-SIC']]
                       .rename(columns={'2017 NAICS US':'naics_code', 'ISIC 4.0':'isic4_code', 'US-SIC':'sic'}))

In [13]:
concordance_df = pd.concat([concordance_2012_df, concordance_2017_df]).reset_index()
concordance_df

,index,naics_code,naics_version,isic4_code,sic
0,0,111110,2012,111,111
1,1,111110,2012,111,112
2,2,111110,2012,111,115
3,3,111110,2012,111,116
4,4,111110,2012,111,119
...,...,...,...,...,...
68158,34039,926110,2017,8412,9531
68159,34040,923130,2017,8430,6321
68160,34041,923130,2017,8430,6324
68161,34042,923130,2017,8430,9441


In [14]:
concordance_df.to_sql("naics_isic_sic", con=engine, schema=ingest_schema,
                      if_exists="append", index=False,
                      method=osc.TrinoBatchInsert(batch_size = 10000, verbose = False))